In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
# !unzip "/content/drive/MyDrive/open.zip"

In [ ]:
# 사용 가능한 GPU 확인
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

# GPU 사용 설정
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
import sys
# sys.path.append('/content/drive/MyDrive/Colab Notebooks/')
sys.path.append(os.path.join(os.getcwd(), os.path.pardir, 'functions'))

from get_train_list import get_train_image_paths_and_encoding_labels_fn
from predict_and_test import get_predict_result_fn, get_test_csv_fn
from get_img_tensor_v2 import get_tensor_through_imgs_fn

from PIL import ImageFile
from datetime import datetime
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.data import Dataset, experimental
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout, BatchNormalization

In [3]:
# ======== 학습용 이미지 경로와 라벨에 대한 각각의 리스트 생성하기 ========

train_image_paths, encoding_labels = get_train_image_paths_and_encoding_labels_fn()

# =====================================================================

encoding_labels.shape : (3457, 19)


In [4]:
# ========= 학습용 이미지 로드, 정규화, 사이즈 조정 =========

# 잘린 이미지에 대한 로드 여부 설정
ImageFile.LOAD_TRUNCATED_IMAGES = True

# ======== AlexNet 코드 ========
RESIZED_WIDTH = 227
RESIZED_HEIGHT = 227
# ==============================

train_image_tensor = get_tensor_through_imgs_fn(train_image_paths, RESIZED_WIDTH, RESIZED_HEIGHT)
print('train_image_tensor.shape :', train_image_tensor.shape)

# ========================================================

100%|██████████| 3457/3457 [01:03<00:00, 54.14it/s]


train_image_tensor.shape : (3457, 227, 227, 3)


In [5]:
# ========================== Xception + AlexNet ==========================

# https://techblog-history-younghunjo1.tistory.com/261
# https://towardsdatascience.com/implementing-alexnet-cnn-architecture-using-tensorflow-2-0-and-keras-2113e090ad98

# Xception + AlexNet (ImageDataGenerator 객체 미사용)
# 데이콘 리더 보드 점수 => 0.3871364552

# TRAIN_FOLDER_PATH = '/content/train/'
TRAIN_FOLDER_PATH = '../../open/train/'
OPEN_FOLDER_PATH = '../../open'

# ======== AlexNet 코드 ========
EPOCHS = 50

train_ds = Dataset.from_tensor_slices(
    (train_image_tensor, encoding_labels))

train_ds_size = experimental.cardinality(train_ds).numpy()
print("Training data size:", train_ds_size)

train_ds = (train_ds
            .shuffle(buffer_size=train_ds_size)
            .batch(batch_size=32, drop_remainder=True))
# ==============================

def create_model_fn():

    input_tensor = Input(shape=(RESIZED_WIDTH, RESIZED_HEIGHT, 3))

    pretrained_model = Xception(
        input_tensor=input_tensor, include_top=False, weights='imagenet')
    pretrained_output = pretrained_model.output

    x = GlobalAveragePooling2D()(pretrained_output)

    # ======== AlexNet 코드 ========
    x = Dense(units=4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(units=4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(units=19, activation='softmax')(x)
    # ==============================

    model = Model(inputs=input_tensor, outputs=output)
    model.summary()
    return model

model = create_model_fn()

# ======== AlexNet 코드 ========
model.compile(optimizer=SGD(learning_rate=0.001),
              loss='categorical_crossentropy', metrics=['acc'])
# ==============================

train_history = model.fit(train_ds, epochs=EPOCHS)
loss = train_history.history['loss']
print('loss :', loss)

# 연월일_시간
now = datetime.today().strftime('%Y%m%d_%H%M%S')
model.save(f'model_{now}.h5')

# ========================================================================

Training data size: 3457
83683744/83683744 [==============================] - 5s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 227, 227, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 113, 113, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 113, 113, 32  128        ['block1_conv1[0][0]']           
 ati

In [ ]:
# ================== 테스트 파일로 추론해보기 ==================

predict_result = get_predict_result_fn(RESIZED_WIDTH, RESIZED_HEIGHT, model)
get_test_csv_fn(predict_result)

# ============================================================